In [1]:
import os

In [2]:
%pwd

'c:\\Users\\hp\\Desktop\\ML\\project\\Thyroid_Detection\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\hp\\Desktop\\ML\\project\\Thyroid_Detection'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_x_data_path: Path
    test_x_data_path: Path
    train_y_data_path: Path
    test_y_data_path: Path
    model_name: str
    criterion: str
    splitter: str
    max_depth: str
    min_samples_split: int
    min_samples_leaf: int
    min_weight_fraction_leaf: float
    max_features: str
    random_state: int
    max_leaf_nodes: str
    min_impurity_decrease: float
    class_weight: str
    ccp_alpha: float
    target_column: str

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.DecisionTreeClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_x_data_path = config.train_x_data_path,
            test_x_data_path = config.test_x_data_path,
            train_y_data_path = config.train_y_data_path,
            test_y_data_path = config.test_y_data_path,
            model_name = config.model_name,
            criterion= params.criterion,
            splitter = params.splitter,
            max_depth = params.max_depth,
            min_samples_split = params.min_samples_split,
            min_samples_leaf = params.min_samples_leaf,
            min_weight_fraction_leaf = params.min_weight_fraction_leaf,
            max_features = params.max_features,
            random_state = params.random_state,
            max_leaf_nodes = params.max_leaf_nodes,
            min_impurity_decrease = params.min_impurity_decrease,
            class_weight = params.class_weight,
            ccp_alpha = params.ccp_alpha,
            target_column = schema.name
            
        )

        return model_trainer_config

In [8]:
import pandas as pd
import os
from mlProject import logger
from sklearn.tree import DecisionTreeClassifier
import joblib

In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_x = pd.read_csv(self.config.train_x_data_path)
        train_y = pd.read_csv(self.config.train_y_data_path)
 

        dt = DecisionTreeClassifier(criterion= self.config.criterion,splitter= self.config.splitter,random_state = self.config.random_state,ccp_alpha=self.config.ccp_alpha)
        dt.fit(train_x, train_y)

        joblib.dump(dt, os.path.join(self.config.root_dir, self.config.model_name))

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2023-11-06 10:01:12,912: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-06 10:01:12,916: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-06 10:01:12,932: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-11-06 10:01:12,932: INFO: common: created directory at: artifacts]
[2023-11-06 10:01:12,938: INFO: common: created directory at: artifacts/model_trainer]
